In [2]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from category_encoders import TargetEncoder, one_hot
%matplotlib inline

In [3]:
data = pd.read_csv('train.csv')
data.dropna(inplace=True)
data.head()

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X13,X14,X15,X16,X17,X18,X19,X20,X21,target
0,0,2,908749,954,480270,935,71,79,53,89,...,51,100,87,3,61,48,43,79,329,0
1,1,1,438444,2162,486685,2154,32,39,48,49,...,39,83,91,6,57,94,15,89,581,1
2,2,1,596915,2066,711059,2081,43,21,58,94,...,7,52,37,50,69,60,89,10,548,0
3,3,1,625198,1508,66810,1474,7,1,34,73,...,39,27,55,97,83,1,88,87,555,1
4,4,2,228654,1202,542816,1196,53,35,16,80,...,35,95,50,55,10,49,12,68,651,0


In [4]:
data.describe()

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X13,X14,X15,X16,X17,X18,X19,X20,X21,target
count,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,...,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06,3.633005e+06
mean,1.816502e+06,1.650469e+00,5.638035e+05,1.449992e+03,5.637574e+05,1.450097e+03,5.016512e+01,5.016514e+01,5.018091e+01,5.015996e+01,...,5.017506e+01,5.017908e+01,5.018681e+01,5.018890e+01,5.014741e+01,5.018218e+01,5.016238e+01,5.015347e+01,4.913625e+02,3.800829e-01
std,1.048758e+06,7.179981e-01,3.248814e+05,4.848098e+02,3.247859e+05,4.848208e+02,2.964439e+01,2.966132e+01,2.963772e+01,2.964519e+01,...,2.965711e+01,2.965469e+01,2.964957e+01,2.965893e+01,2.965769e+01,2.965730e+01,2.963982e+01,2.966294e+01,1.464528e+02,4.854070e-01
min,0.000000e+00,1.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00,0.000000e+00
25%,9.082510e+05,1.000000e+00,2.829920e+05,1.074000e+03,2.829370e+05,1.074000e+03,2.400000e+01,2.400000e+01,2.400000e+01,2.400000e+01,...,2.400000e+01,2.400000e+01,2.400000e+01,2.400000e+01,2.400000e+01,2.400000e+01,2.400000e+01,2.400000e+01,3.840000e+02,0.000000e+00
50%,1.816502e+06,2.000000e+00,5.637460e+05,1.370000e+03,5.639180e+05,1.371000e+03,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,...,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.350000e+02,0.000000e+00
75%,2.724753e+06,2.000000e+00,8.454180e+05,1.747000e+03,8.454590e+05,1.747000e+03,7.900000e+01,7.900000e+01,7.900000e+01,7.900000e+01,...,7.900000e+01,7.900000e+01,7.900000e+01,7.900000e+01,7.900000e+01,7.900000e+01,7.900000e+01,7.900000e+01,6.150000e+02,1.000000e+00
max,3.633004e+06,8.000000e+00,1.126077e+06,4.186000e+03,1.126076e+06,4.192000e+03,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,...,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,6.800000e+02,1.000000e+00


In [29]:
#target_encoder_X2 = TargetEncoder()
#target_encoder_X2.fit(data['X2'], data['target'])

# target_encoder_X4 = TargetEncoder()
# target_encoder_X4.fit(data['X4'], data['target'])

# data['X2'] = target_encoder_X2.transform(data['X2'])
# data['X4'] = target_encoder_X4.transform(data['X4'])
# data.head()

In [40]:
#dummies_X0 = pd.get_dummies(data['X0'])
#data_train = pd.concat([data, dummies_X0], axis=1)

# pd.get_dummies(data.loc[:, 'X0'], prefix='X0')

In [11]:
categorical_features = ['X0', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10']

df = data[['X1', 'X2', 'X3', 'X4', 'X21']]
for cat_names in categorical_features:
    dummies = pd.get_dummies(data.loc[:, cat_names], prefix=cat_names, sparse=True)
    df = pd.concat([df, dummies], axis=1)

In [12]:
df

,X1,X2,X3,X4,X21,X0_1,X0_2,X0_3,X0_4,X0_5,...,X10_91,X10_92,X10_93,X10_94,X10_95,X10_96,X10_97,X10_98,X10_99,X10_100
0,908749,954,480270,935,329,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,438444,2162,486685,2154,581,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,596915,2066,711059,2081,548,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,625198,1508,66810,1474,555,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,228654,1202,542816,1196,651,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3633000,644871,1211,421763,1194,615,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3633001,689210,699,569184,706,517,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3633002,227914,1063,174332,1062,279,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3633003,1066558,1752,358523,1780,669,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [13]:
X, y = df, data.loc[:, 'target']

In [14]:
print(f'X shape = {X.shape}','\n', f'y shape = {y.shape}')

X shape = (3633005, 618) 
 y shape = (3633005,)


In [21]:
X = X.values
y = y.values

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=241, shuffle=True)

MemoryError: Unable to allocate 3.35 GiB for an array with shape (726601, 618) and data type int64

In [16]:
cls_rf = RandomForestClassifier(random_state=241, n_estimators=150, bootstrap=True, n_jobs=2)
cls_rf.fit(X_train, y_train)

c:\users\kanygin.yy\desktop\ozone-games\.venv\lib\site-packages\sklearn\utils\validation.py:572: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


RandomForestClassifier(n_estimators=150, n_jobs=2, random_state=241)

In [18]:
log_loss(y_test, cls_rf.predict_proba(X_test))

c:\users\kanygin.yy\desktop\ozone-games\.venv\lib\site-packages\sklearn\utils\validation.py:572: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


0.6699354570617579

In [ ]:
l_rates = [1, 0.5, 0.3, 0.2, 0.1]

mins_train = []
argmins_train = []
mins_test = []
argmins_test = []

for learning_rate in l_rates:
    model = GradientBoostingClassifier(
        n_estimators=150,
        verbose=False,
        random_state=241,
        learning_rate=learning_rate
        )
    model.fit(X_train, y_train)

    test_score = list(range(250))
    train_score = list(range(250))

    for i, predictions in enumerate(model.staged_decision_function(X_test)):
        predictions = [x[0] for x in predictions.tolist()] # unpack this stupid format
        predictions = [1/(1 + np.exp(-x)) for x in predictions]
        test_score[i] = log_loss(y_test, predictions)

    for i, predictions in enumerate(model.staged_decision_function(X_train)):
        predictions = [x[0] for x in predictions.tolist()] # unpack this stupid format
        predictions = [1/(1 + np.exp(-x)) for x in predictions]
        train_score[i] = log_loss(y_train, predictions)
        
    mins_train.append(np.min(train_score))
    argmins_train.append(np.argmin(train_score))
    mins_test.append(np.min(test_score))
    argmins_test.append(np.argmin(test_score))
        
    plt.figure()
    plt.plot(test_score, 'r', linewidth=2)
    plt.plot(train_score, 'g', linewidth=2)
    plt.legend(['test', 'train'])

In [ ]:
print(mins_test, argmins_test)

In [47]:
data_test = pd.read_csv('test.csv')
data_test.head()

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21
0,0,1,1032217,1187,1011523,1172,57,100,1,80,...,79,45,18,35,10,80,81,16,0,547
1,1,2,1059033,2128,505263,2104,82,27,87,79,...,74,96,55,54,48,3,92,5,51,656
2,2,1,207787,1017,822639,1040,82,48,43,62,...,16,64,35,70,99,60,30,37,45,437
3,3,1,1095582,1449,32841,1424,71,43,11,98,...,1,7,41,94,26,52,45,81,1,589
4,4,2,963764,1202,489871,1182,11,62,16,55,...,26,60,37,7,1,80,48,50,52,340


In [50]:
X_final_test = data_test.loc[:, 'X0':'X21']

X_final_test['X2'] = target_encoder_X2.transform(X_final_test['X2'])
X_final_test['X4'] = target_encoder_X4.transform(X_final_test['X4'])

dummies_X0 = pd.get_dummies(X_final_test['X0'])
X_final_test = pd.concat([X_final_test, dummies_X0], axis=1)
X_final_test = X_final_test.loc[:, 'X1':]
X_final_test.head() 

d:\ozone-games\.venv\lib\site-packages\pandas\core\frame.py:3599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item_frame_value(key, value)


,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X19,X20,X21,1,2,3,4,5,6,8
0,1032217,1187,1011523,1172,57,100,1,80,25,87,...,16,0,547,1,0,0,0,0,0,0
1,1059033,2128,505263,2104,82,27,87,79,80,71,...,5,51,656,0,1,0,0,0,0,0
2,207787,1017,822639,1040,82,48,43,62,18,60,...,37,45,437,1,0,0,0,0,0,0
3,1095582,1449,32841,1424,71,43,11,98,80,33,...,81,1,589,1,0,0,0,0,0,0
4,963764,1202,489871,1182,11,62,16,55,88,92,...,50,52,340,0,1,0,0,0,0,0


In [52]:
y_final_test = cls_rf2.predict_proba(X_final_test)
y_final_test = y_final_test[:, 1]

In [53]:
ans = pd.DataFrame(data=y_final_test, columns=["target"])
ans.reset_index(level=0, inplace=True)
ans.reset_index(drop=True, inplace=True)
ans = ans.rename(columns={'index':'id'})
ans.head()

,id,target
0,0,0.420000
1,1,0.386667
2,2,0.366667
3,3,0.433333
4,4,0.466667


In [54]:
ans.to_csv('submission.csv',index=False)